Librería con los métodos que se encargan de calcular la **distancia de compresión normalizada (NCD) entre dos noticias** y obtener la **cantidad de información que ha extraído una noticia de la otra**.

Métodos para el **cálculo de la NCD**:

In [2]:
# Librerias necesarias
import gzip
import os
import sys

Cuando se calcula la **NCD entre dos noticias**, debido a que el compresor **GZIP no es del todo simétrico**, es necesario calcular la **NCD(noticia1, noticia2)** y **NCD(noticia2, noticia1)** ya que **no es una opración completamente simétrica**:

**NCD(noticia1, noticia2) = ( C(noticia1noticia2) - C(noticia1) ) / C(noticia2)**  -> La noticia2 ha derivado de la noticia1

**NCD(noticia2, noticia1) = ( C(noticia2noticia1) - C(noticia2) ) / C(noticia1)**  -> La noticia1 ha derivado de la noticia2

Si la NCD(noticia1, noticia2) es menor, entonces la noticia2 ha derivado de la noticia1. Por el contrario, la noticia1 ha dereivado de la noticia2.

**C(fichero) = longitud (len) del contenido del fichero comprimido.**

Métodos para leer y escribir un fichero:

In [3]:
# Lee el contenido de un fichero y lo devuelve
def read_file(filename):
    try:
        f = open(filename, "rb")
    except IOError as e:
        print(e.errno, e.message)
    else:
        data = f.read()
        f.close()
        return data

# Escribe el contenido en bytes en un fichero
def write_file(filename, data):
    try:
        f = open(filename, "wb")
    except IOError as e:
        print(e.errno, e.message)
    else:
        f.write(data)
        f.close()

# Escribe el contenido en un fichero        
def write_file_str(filename, data):
    try:
        f = open(filename, "w", encoding='utf-8')
    except IOError as e:
        print(e.errno, e.message)
    else:
        f.write(data)
        f.close()

Métodos para comprimir/descomprimir con GZIP:

In [4]:
# Comprime el fichero con GZIP
def compress(filename):
    data = read_file(filename)
    
    if data is not None:
        f = gzip.open(filename + ".gz", "wb")
        f.write(data)
        f.close()

# Descomprime el fichero con GZIP
def decompress(filename):
    f = gzip.open(filename)
    write_file(filename[:filename.rfind(".gz")], f.read())
    f.close()

**Función** que a partir de **dos ficheros de texto de noticias** me devuelve **[NCD12, NCD21]**:

In [6]:
# Calcula la NCD en los dos sentidos a partir de dos ficheros con los contenidos de las noticias
def NCD(filename_1, filename_2):
    filename_12 = "noticia_12.txt" # Fichero para concatenar la noticia1 y la noticia2, en este orden
    filename_21 = "noticia_21.txt" # Fichero para concatenar la noticia2 y la noticia1, en este orden

    # Se concatenan los contenidos de ambos ficheros
    write_file(filename_12, read_file(filename_1) + read_file(filename_2)) # Primero el contenido de la noticia1 
    write_file(filename_21, read_file(filename_2) + read_file(filename_1)) # Primero el contenido de la noticia2 
    
    # Se comprimen todos los ficheros con GZIP
    compress(filename_1) # Fichero con el contenido de la noticia1
    compress(filename_2) # Fichero con el contenido de la noticia2
    compress(filename_12) # Fichero con el contenido de la noticia1 concatenado con la noticia2
    compress(filename_21) # Fichero con el contenido de la noticia2 concatenado con la noticia 1

    # Nombres de los ficheros comprimidos generados
    filename_1_compress = filename_1 + ".gz"
    filename_2_compress = filename_2 + ".gz"
    filename_12_compress = filename_12 + ".gz"
    filename_21_compress = filename_21 + ".gz"
    
    # Calculo de la NCD en ambos sentidos siguiendo la formula
    NCD12 = ( len(read_file(filename_12_compress)) - len(read_file(filename_1_compress)) ) / len(read_file(filename_2_compress))
    NCD21 = ( len(read_file(filename_21_compress)) - len(read_file(filename_2_compress)) ) / len(read_file(filename_1_compress))
    
    # Se devuelve un array con ambas NCD
    return [NCD12,NCD21]

Método para obtener la **cantidad de información**:

**¿Cómo se calcula?**

Si C(noticia2|noticia1) es la **información que hay en la noticia2 que no tiene nada que ver con la noticia 1**:

   **C(noticia2|noticia1) = C(noticia1noticia2) - C(noticia1)**
   
Entonces si a C(noticia2) le quitamos C(noticia2|noticia1) obtenemos **la cantidad de información que hay en la noticia2 que proviene de la noticia 1**:

   **cantidad_info_n1_en_n2 = C(noticia2) - C(noticia2|noticia1)**
    
Es decir, nos queda:

   **cantidad_info_n1_en_n2 = C(noticia2) - C(noticia1noticia2) + C(noticia1)**

Función que a partir de dos ficheros, con el contenido de las noticias, me **devuelve la cantidad de información que se ha 
cogido de la primera noticia (filename_1) para añadirla en la segunda noticia (filename_2).**

In [7]:
# Metodo que devuelve la cantidad de información que ha cogido la noticia2 de la noticia1
def cantidad_info_n1_en_n2(filename_1, filename_2):
    filename_12 = "noticia_12.txt" # Fichero para concatenar la noticia1 con la 2, en este orden

    write_file(filename_12, read_file(filename_1) + read_file(filename_2)) # Escribe en un fichero la concatenacion de 1 con 2
    
    # Comprimo los ficheros
    compress(filename_1) # Comprimo el fichero con el contenido de la noticia1
    compress(filename_2) # Comprimo el fichero con el contenido de la noticia2
    compress(filename_12) # Comprimo el fichero con el contenido de la noticia1 concatenada con 2

    # Nombre de los ficheros comprimidos generados
    filename_1_compress = filename_1 + ".gz"
    filename_2_compress = filename_2 + ".gz"
    filename_12_compress = filename_12 + ".gz"
    
    # Calculo la cantidad de informacion siguiendo la formula
    cant = len(read_file(filename_2_compress)) - len(read_file(filename_12_compress)) + len(read_file(filename_1_compress))
    
    return cant